# ResNet
CNN: 흑백 사진 <br>
컬러 데이터셋 사용 :ResNet

- 신경망을 깊게 쌓으면 오히려 성능이 나빠짐으로 이의 문제 해결, DenseNet등의 파생모델에 영향
- 컨볼루션 층의 출력에 전의 전 계층에 쓰였던 입력을 더함으로써 특징이 유실되지 않도록 함

In [ ]:
import torch
import torch.nn as nn
import torch.optim  as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
EPOCHS = 300
BATCH_SIZE = 128

CIFAR-10 : 10개의 이미지 분류
- R,G,B: 8bit씩 (0~225) 3채널
- PNG는 투명도까지 4종류의 채널

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train = True,
                    download = True,
                    transform = transforms.Compose([
                        transforms.RandomCrop(32, padding = 4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5,0.5,0.5))])),
        batch_size = BATCH_SIZE, shuffle = True
)
#과적합 방지하기 위하여 RandomCrop, RandomHorizontalFlip 사용

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train = False,
                    transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5,0.5,0.5))])),
        batch_size = BATCH_SIZE, shuffle = True
)

Extracting ./.data\cifar-10-python.tar.gz to ./.data


#### Residual 블록의 출력에 입력이었던 x를 더함으로써 모델을 훨씬 깊게 설계할 수 있도록 함

**배치 정규화**
- 학습률을 너무 높게 잡으면 기울기가 소실되거나 발산하는 증상을 예방하여 학습과정을 안정화하는 방법
- 각 계층에 들어가는 입력을 평균과 분산으로 정규화 함으로써 학습을 효율적으로 만들어줌
    - 드롭아웃과 같은 효과를 냄

In [23]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias =False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(planes)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn1(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [24]:
class ResNet(nn.Module):
    def __init__(self, num_classes =10):
        super(ResNet, self).__init__()
        self.in_planes = 16 #layer 1층이 입력 받는 채널의 수
        self.conv1 = nn.Conv2d(3,16, kernel_size =3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16,2, stride = 1)
        self.layer2 = self._make_layer(32,2, stride = 2)
        self.layer3 = self._make_layer(64,2, stride = 2)
        self.linear = nn.Linear(64, num_classes)
    def _make_layer(self, planes, num_blocks, stride):
        stride = [stride] + [1]*(num_blocks - 1)
        layers = []
        for stride in stride:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [25]:
model = ResNet().to(DEVICE)
optimizer= optim.SGD(model.parameters(), lr = 0.1, momentum =0.9, weight_decay = 0.0005)
scheduler  = optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma = 0.1)